## Params and loading packages

In [1]:
## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']



In [2]:

import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

('Number of tissues: ', 16)


In [ ]:
from Modules.Homer import *

In [ ]:
print('Number of tissues: ',len(tissues))

### Load in meta_sampe

In [3]:
meta_samples = pd.read_csv('Results/meta_samples.csv',index_col=0)
meta_samples.head()

,Experiment,Full,Short,Short_with_number,Tissue
BMDMwt/GRO/f04_peaks/merge_bg_2.peak,GRO,/data/isshamie/TSS/Processed/BMDMwt/GRO/f04_pe...,BMDMwt_GRO,BMDMwt_GRO1,BMDMwt
Brain/GRO/f04_peaks/merge_bg_2.peak,GRO,/data/isshamie/TSS/Processed/Brain/GRO/f04_pea...,Brain_GRO,Brain_GRO1,Brain
Brain/mSTART/f04_peaks/merge_bg_2.peak,mSTART,/data/isshamie/TSS/Processed/Brain/mSTART/f04_...,Brain_mSTART,Brain_mSTART1,Brain
Heart/mSTART/f04_peaks/merge_bg_2.peak,mSTART,/data/isshamie/TSS/Processed/Heart/mSTART/f04_...,Heart_mSTART,Heart_mSTART1,Heart
Kidney/GRO/f04_peaks/merge_bg_2.peak,GRO,/data/isshamie/TSS/Processed/Kidney/GRO/f04_pe...,Kidney_GRO,Kidney_GRO1,Kidney


# Create tag directories of RNA

In [4]:
for f in glob.glob(os.path.join(data_folder ,'RNA_Total','*bam')):
    curr_f = os.path.basename(f).split('.')[0]
    out = os.path.join(os.path.join(data_folder ,'RNA_Total',curr_f))
    
    cmd = 'makeTagDirectory {out} -single {in_f} '.format(out=out,in_f=f)
    print(cmd)
    !{cmd}

makeTagDirectory /data/isshamie/TSS/Processed/RNA_Total/NL3_Spleen_S3_L006_R1_001 -single /data/isshamie/TSS/Processed/RNA_Total/NL3_Spleen_S3_L006_R1_001.fastq.bam 
	Will parse file: /data/isshamie/TSS/Processed/RNA_Total/NL3_Spleen_S3_L006_R1_001.fastq.bam

	Creating directory: /data/isshamie/TSS/Processed/RNA_Total/NL3_Spleen_S3_L006_R1_001 and removing existing *.tags.tsv

	Treating /data/isshamie/TSS/Processed/RNA_Total/NL3_Spleen_S3_L006_R1_001.fastq.bam as a bam file
	Reading alignment file /data/isshamie/TSS/Processed/RNA_Total/NL3_Spleen_S3_L006_R1_001.fastq.bam

	Optimizing tag files...
	Optimizing single genome.tags.tsv file...
	Estimated genome size = 2328016001
	Total Tags = 25459523.0
	Total Positions = 34650336
	Average tag length = 92.8
	Median tags per position = 0 (ideal: 1)
	Average tags per position = 0.293
	Fragment Length Estimate: 130
	Peak Width Estimate: 0
		!!! No reliable estimate for peak size
		Setting Peak width estimate to be equal to fragment length esti


	Optimizing tag files...
	Optimizing single genome.tags.tsv file...
	Estimated genome size = 2329212908
	Total Tags = 22125635.0
	Total Positions = 31513051
	Average tag length = 92.2
	Median tags per position = 0 (ideal: 1)
	Average tags per position = 0.256
	Fragment Length Estimate: 130
	Peak Width Estimate: 0
		!!! No reliable estimate for peak size
		Setting Peak width estimate to be equal to fragment length estimate
	Autocorrelation quality control metrics:
		Same strand fold enrichment: 1.5
		Diff strand fold enrichment: 1.6
		Same / Diff fold enrichment: 1.0

		Guessing sample is ChIP-Seq or unstranded RNA-Seq - autocorrelation looks good.

makeTagDirectory /data/isshamie/TSS/Processed/RNA_Total/NL10_Misc_S10_L006_R1_001 -single /data/isshamie/TSS/Processed/RNA_Total/NL10_Misc_S10_L006_R1_001.fastq.bam 
	Will parse file: /data/isshamie/TSS/Processed/RNA_Total/NL10_Misc_S10_L006_R1_001.fastq.bam

	Creating directory: /data/isshamie/TSS/Processed/RNA_Total/NL10_Misc_S10_L006_R1_

## Merge replicates (in this case Brain)

In [75]:
curr_out = os.path.join(data_folder,'RNA_Total')
cmd = 'makeTagDirectory {out_f} -single -d {rep1} {rep2}'.format(out_f=os.path.join(curr_out,'Combined_Brain'),
                                                        rep1=os.path.join(curr_out,'NL11_Brain7-1_S11_L006_R1_001'),
                                                        rep2=os.path.join(curr_out,'NL7_Brain2_S7_L006_R1_001'))
print(cmd)
!{cmd}

makeTagDirectory /data/isshamie/TSS/Processed/RNA_Total/Combined_Brain -single -d /data/isshamie/TSS/Processed/RNA_Total/NL11_Brain7-1_S11_L006_R1_001 /data/isshamie/TSS/Processed/RNA_Total/NL7_Brain2_S7_L006_R1_001
	Will add tag directory: /data/isshamie/TSS/Processed/RNA_Total/NL11_Brain7-1_S11_L006_R1_001
	Will add tag directory: /data/isshamie/TSS/Processed/RNA_Total/NL7_Brain2_S7_L006_R1_001

	Creating directory: /data/isshamie/TSS/Processed/RNA_Total/Combined_Brain and removing existing *.tags.tsv
	Adding tag directory /data/isshamie/TSS/Processed/RNA_Total/NL11_Brain7-1_S11_L006_R1_001
	Adding tag directory /data/isshamie/TSS/Processed/RNA_Total/NL7_Brain2_S7_L006_R1_001

	Optimizing tag files...
	Optimizing single genome.tags.tsv file...
	Estimated genome size = 2338237273
	Total Tags = 50968641.0
	Total Positions = 69562139
	Average tag length = 92.4
	Median tags per position = 0 (ideal: 1)
	Average tags per position = 0.297
	Fragment Length Estimate: 120
	Peak Width Estimate:

### Also merge every file together for later use

In [234]:
curr_out = os.path.join(data_folder,'RNA_Total/*')
files = glob.glob(curr_out)
rna_dirs = []
for f in files:
    if os.path.isdir(f) and 'Combined_Brain' not in f:
        rna_dirs.append(f)
cmd = 'makeTagDirectory {out_f} -single -d {all_rna}'.format(out_f='Results/merged/tags_rna_merged',
                                                        all_rna= ' '.join(rna_dirs))
print(cmd)
!{cmd}

makeTagDirectory Results/merged/tags_rna_merged -single -d /data/isshamie/TSS/Processed/RNA_Total/NL10_Misc_S10_L006_R1_001 /data/isshamie/TSS/Processed/RNA_Total/NL11_Brain7-1_S11_L006_R1_001 /data/isshamie/TSS/Processed/RNA_Total/NL3_Spleen_S3_L006_R1_001 /data/isshamie/TSS/Processed/RNA_Total/NL4_Lung_S4_L006_R1_001 /data/isshamie/TSS/Processed/RNA_Total/NL8_Muscle_S8_L006_R1_001 /data/isshamie/TSS/Processed/RNA_Total/NL2_ReproductiveTract_S2_L006_R1_001 /data/isshamie/TSS/Processed/RNA_Total/NL5_Liver_S5_L006_R1_001 /data/isshamie/TSS/Processed/RNA_Total/NL7_Brain2_S7_L006_R1_001
	Will add tag directory: /data/isshamie/TSS/Processed/RNA_Total/NL10_Misc_S10_L006_R1_001
	Will add tag directory: /data/isshamie/TSS/Processed/RNA_Total/NL11_Brain7-1_S11_L006_R1_001
	Will add tag directory: /data/isshamie/TSS/Processed/RNA_Total/NL3_Spleen_S3_L006_R1_001
	Will add tag directory: /data/isshamie/TSS/Processed/RNA_Total/NL4_Lung_S4_L006_R1_001
	Will add tag directory: /data/isshamie/TSS/Pro

# Create bedgraph files for RNA

In [70]:
for f in glob.glob(os.path.join(data_folder ,'RNA_Total','*bam')):
    curr_f = os.path.basename(f).split('.')[0]
    out = os.path.join(os.path.join(data_folder ,'RNA_Total',curr_f))

    cmd = 'makeUCSCfile {tag_d} -fragLength given -o auto'.format(tag_d=out)
    print(cmd)
    !{cmd}



makeUCSCfile /data/isshamie/TSS/Processed/RNA_Total/NL3_Spleen_S3_L006_R1_001 -fragLength given -o auto

	Visualization fragment length = given
	Output file: /data/isshamie/TSS/Processed/RNA_Total/NL3_Spleen_S3_L006_R1_001/NL3_Spleen_S3_L006_R1_001.ucsc.bedGraph
	No need to remove tags to get desired file size
	Generating bedGraph for picr_0
	Generating bedGraph for picr_1
	Generating bedGraph for picr_10
	Generating bedGraph for picr_100
	Generating bedGraph for picr_1000
	Generating bedGraph for picr_1002
	Generating bedGraph for picr_1003
	Generating bedGraph for picr_1005
	Generating bedGraph for picr_1006
	Generating bedGraph for picr_1008
	Generating bedGraph for picr_1009
	Generating bedGraph for picr_101
	Generating bedGraph for picr_1010
	Generating bedGraph for picr_1015
	Generating bedGraph for picr_1018
	Generating bedGraph for picr_102
	Generating bedGraph for picr_1021
	Generating bedGraph for picr_1022
	Generating bedGraph for picr_1023
	Generating bedGraph for picr_1028

	Generating bedGraph for picr_1311
	Generating bedGraph for picr_1313
	Generating bedGraph for picr_1314
	Generating bedGraph for picr_1315
	Generating bedGraph for picr_1316
	Generating bedGraph for picr_1317
	Generating bedGraph for picr_1318
	Generating bedGraph for picr_132
	Generating bedGraph for picr_1320
	Generating bedGraph for picr_1322
	Generating bedGraph for picr_1323
	Generating bedGraph for picr_1324
	Generating bedGraph for picr_1326
	Generating bedGraph for picr_1327
	Generating bedGraph for picr_1328
	Generating bedGraph for picr_1329
	Generating bedGraph for picr_133
	Generating bedGraph for picr_1330
	Generating bedGraph for picr_1332
	Generating bedGraph for picr_1335
	Generating bedGraph for picr_1336
	Generating bedGraph for picr_1337
	Generating bedGraph for picr_1338
	Generating bedGraph for picr_1339
	Generating bedGraph for picr_134
	Generating bedGraph for picr_1340
	Generating bedGraph for picr_1341
	Generating bedGraph for picr_1344
	Generating bedGraph fo

	Generating bedGraph for picr_1660
	Generating bedGraph for picr_1663
	Generating bedGraph for picr_1664
	Generating bedGraph for picr_1666
	Generating bedGraph for picr_1668
	Generating bedGraph for picr_1669
	Generating bedGraph for picr_167
	Generating bedGraph for picr_1670
	Generating bedGraph for picr_1672
	Generating bedGraph for picr_1673
	Generating bedGraph for picr_1674
	Generating bedGraph for picr_1675
	Generating bedGraph for picr_1677
	Generating bedGraph for picr_1678
	Generating bedGraph for picr_168
	Generating bedGraph for picr_1680
	Generating bedGraph for picr_1683
	Generating bedGraph for picr_1684
	Generating bedGraph for picr_1685
	Generating bedGraph for picr_1687
	Generating bedGraph for picr_1688
	Generating bedGraph for picr_169
	Generating bedGraph for picr_1699
	Generating bedGraph for picr_17
	Generating bedGraph for picr_170
	Generating bedGraph for picr_1700
	Generating bedGraph for picr_1701
	Generating bedGraph for picr_1702
	Generating bedGraph for p

	Generating bedGraph for picr_330
	Generating bedGraph for picr_331
	Generating bedGraph for picr_332
	Generating bedGraph for picr_333
	Generating bedGraph for picr_334
	Generating bedGraph for picr_336
	Generating bedGraph for picr_337
	Generating bedGraph for picr_338
	Generating bedGraph for picr_339
	Generating bedGraph for picr_34
	Generating bedGraph for picr_341
	Generating bedGraph for picr_342
	Generating bedGraph for picr_343
	Generating bedGraph for picr_346
	Generating bedGraph for picr_347
	Generating bedGraph for picr_348
	Generating bedGraph for picr_349
	Generating bedGraph for picr_35
	Generating bedGraph for picr_350
	Generating bedGraph for picr_351
	Generating bedGraph for picr_352
	Generating bedGraph for picr_353
	Generating bedGraph for picr_354
	Generating bedGraph for picr_355
	Generating bedGraph for picr_356
	Generating bedGraph for picr_357
	Generating bedGraph for picr_358
	Generating bedGraph for picr_359
	Generating bedGraph for picr_36
	Generating bedGr

	Generating bedGraph for picr_591
	Generating bedGraph for picr_592
	Generating bedGraph for picr_594
	Generating bedGraph for picr_595
	Generating bedGraph for picr_596
	Generating bedGraph for picr_597
	Generating bedGraph for picr_598
	Generating bedGraph for picr_6
	Generating bedGraph for picr_60
	Generating bedGraph for picr_601
	Generating bedGraph for picr_602
	Generating bedGraph for picr_603
	Generating bedGraph for picr_605
	Generating bedGraph for picr_606
	Generating bedGraph for picr_607
	Generating bedGraph for picr_608
	Generating bedGraph for picr_609
	Generating bedGraph for picr_61
	Generating bedGraph for picr_610
	Generating bedGraph for picr_612
	Generating bedGraph for picr_613
	Generating bedGraph for picr_615
	Generating bedGraph for picr_616
	Generating bedGraph for picr_619
	Generating bedGraph for picr_62
	Generating bedGraph for picr_620
	Generating bedGraph for picr_622
	Generating bedGraph for picr_623
	Generating bedGraph for picr_624
	Generating bedGrap

...

	Generating bedGraph for picr_1261
	Generating bedGraph for picr_1262
	Generating bedGraph for picr_1263
	Generating bedGraph for picr_1264
	Generating bedGraph for picr_1265
	Generating bedGraph for picr_1266
	Generating bedGraph for picr_1268
	Generating bedGraph for picr_1269
	Generating bedGraph for picr_127
	Generating bedGraph for picr_1270
	Generating bedGraph for picr_1275
	Generating bedGraph for picr_1277
	Generating bedGraph for picr_1278
	Generating bedGraph for picr_128
	Generating bedGraph for picr_1281
	Generating bedGraph for picr_1283
	Generating bedGraph for picr_1285
	Generating bedGraph for picr_1286
	Generating bedGraph for picr_1287
	Generating bedGraph for picr_1288
	Generating bedGraph for picr_1289
	Generating bedGraph for picr_129
	Generating bedGraph for picr_1292
	Generating bedGraph for picr_1293
	Generating bedGraph for picr_1294
	Generating bedGraph for picr_1295
	Generating bedGraph for picr_1296
	Generating bedGraph for picr_1297
	Generating bedGraph fo

	Generating bedGraph for picr_1601
	Generating bedGraph for picr_1603
	Generating bedGraph for picr_1605
	Generating bedGraph for picr_161
	Generating bedGraph for picr_1611
	Generating bedGraph for picr_1614
	Generating bedGraph for picr_1615
	Generating bedGraph for picr_162
	Generating bedGraph for picr_1621
	Generating bedGraph for picr_1623
	Generating bedGraph for picr_1625
	Generating bedGraph for picr_1627
	Generating bedGraph for picr_1628
	Generating bedGraph for picr_1629
	Generating bedGraph for picr_163
	Generating bedGraph for picr_1631
	Generating bedGraph for picr_1632
	Generating bedGraph for picr_1633
	Generating bedGraph for picr_1634
	Generating bedGraph for picr_164
	Generating bedGraph for picr_1640
	Generating bedGraph for picr_1645
	Generating bedGraph for picr_1646
	Generating bedGraph for picr_1647
	Generating bedGraph for picr_165
	Generating bedGraph for picr_1650
	Generating bedGraph for picr_1651
	Generating bedGraph for picr_1656
	Generating bedGraph for 

	Generating bedGraph for picr_30
	Generating bedGraph for picr_300
	Generating bedGraph for picr_301
	Generating bedGraph for picr_302
	Generating bedGraph for picr_303
	Generating bedGraph for picr_304
	Generating bedGraph for picr_306
	Generating bedGraph for picr_308
	Generating bedGraph for picr_309
	Generating bedGraph for picr_31
	Generating bedGraph for picr_310
	Generating bedGraph for picr_311
	Generating bedGraph for picr_312
	Generating bedGraph for picr_313
	Generating bedGraph for picr_314
	Generating bedGraph for picr_315
	Generating bedGraph for picr_316
	Generating bedGraph for picr_317
	Generating bedGraph for picr_318
	Generating bedGraph for picr_319
	Generating bedGraph for picr_32
	Generating bedGraph for picr_321
	Generating bedGraph for picr_322
	Generating bedGraph for picr_323
	Generating bedGraph for picr_325
	Generating bedGraph for picr_326
	Generating bedGraph for picr_328
	Generating bedGraph for picr_329
	Generating bedGraph for picr_33
	Generating bedGra

	Generating bedGraph for picr_550
	Generating bedGraph for picr_551
	Generating bedGraph for picr_554
	Generating bedGraph for picr_555
	Generating bedGraph for picr_556
	Generating bedGraph for picr_557
	Generating bedGraph for picr_558
	Generating bedGraph for picr_559
	Generating bedGraph for picr_56
	Generating bedGraph for picr_561
	Generating bedGraph for picr_562
	Generating bedGraph for picr_563
	Generating bedGraph for picr_566
	Generating bedGraph for picr_567
	Generating bedGraph for picr_568
	Generating bedGraph for picr_57
	Generating bedGraph for picr_571
	Generating bedGraph for picr_572
	Generating bedGraph for picr_573
	Generating bedGraph for picr_574
	Generating bedGraph for picr_576
	Generating bedGraph for picr_577
	Generating bedGraph for picr_579
	Generating bedGraph for picr_58
	Generating bedGraph for picr_580
	Generating bedGraph for picr_581
	Generating bedGraph for picr_582
	Generating bedGraph for picr_583
	Generating bedGraph for picr_584
	Generating bedGr

	Generating bedGraph for picr_850
	Generating bedGraph for picr_852
	Generating bedGraph for picr_853
	Generating bedGraph for picr_854
	Generating bedGraph for picr_857
	Generating bedGraph for picr_858
	Generating bedGraph for picr_86
	Generating bedGraph for picr_860
	Generating bedGraph for picr_861
	Generating bedGraph for picr_862
	Generating bedGraph for picr_863
	Generating bedGraph for picr_866
	Generating bedGraph for picr_868
	Generating bedGraph for picr_87
	Generating bedGraph for picr_870
	Generating bedGraph for picr_873
	Generating bedGraph for picr_874
	Generating bedGraph for picr_875
	Generating bedGraph for picr_877
	Generating bedGraph for picr_878
	Generating bedGraph for picr_879
	Generating bedGraph for picr_88
	Generating bedGraph for picr_880
	Generating bedGraph for picr_882
	Generating bedGraph for picr_883
	Generating bedGraph for picr_884
	Generating bedGraph for picr_885
	Generating bedGraph for picr_886
	Generating bedGraph for picr_89
	Generating bedGra

------------------------------------

# SessionInfo

In [28]:
pip_modules = !conda list #you could also use `!conda list` with anaconda
#print the names and versions of the imported modules
package_lists = ''
for module in pip_modules:
    if module[0] == '#': ## Comment
        continue
    package_lists = package_lists + module.split(' ')[0] +  ','
package_lists = package_lists.strip(',')
%load_ext version_information
%version_information $package_lists version_information

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


Software,Version
Python,2.7.13 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython,5.3.0
OS,Linux 4.4.0 130 generic x86_64 with debian stretch sid
_license,361fa109f3a40c3f6f5fd26911d452e4a909c634
alabaster,0.7.10
anaconda,The 'anaconda' distribution was not found and is required by the application
anaconda-client,1.6.3
anaconda-navigator,1.6.2
anaconda-project,0.6.0
asn1crypto,0.22.0
